In [54]:
import pymongo
import requests
import json
import numpy as np
import pandas as pd
import time
from tqdm import tqdm
import datetime
from credentials import client_id,client_secret
from ipynb.fs.full.repo_collector import cleanLinks, getOrgNRepo, org_name, repo_name

In [55]:
def addMISCDetails(repo_id, org_id, issue_number, list_details):
    updt_list = []    
    add_details = {"org_id":org_id, "repo_id":repo_id, "issue_id":issue_number}
    for ele in list_details:
        ele.update(add_details)
        updt_list.append(ele)            
    return updt_list

In [60]:
def populateToDB(url, org_name, repo_name, issue_number, db_name):
    
    response = requests.get(url,auth=(client_id, client_secret))
    print(response.headers.get('Link'))
#     print(response.headers.get('X-RateLimit-Remaining'))
    if int(response.headers.get('X-RateLimit-Remaining')) <= 2:
        time.sleep(3600)
        
    if response.json():    
        raw_list = response.json()
    #     print(len(raw_list))

        mongo_client = pymongo.MongoClient("mongodb://127.0.0.1:27017/")
        repodb = mongo_client["repo_collector"]
        repo_coll = repodb[db_name]

    #     print(raw_list)
        repo_id, org_id = getOrgNRepo(org_name,repo_name,"tf_repo")
    #     print(str(repo_id)+" | "+str(org_id))

        updt_list = addMISCDetails(repo_id, org_id, issue_number, raw_list)
        #print(issue_raw_list[-1])

        repo_coll.insert_many(updt_list)

        if response.headers.get('Link'):
            link = response.headers.get('Link')
        #     print(link)
            clean_links = cleanLinks(link)        
        #     print(clean_links)

            df = pd.DataFrame(clean_links, columns =['url', 'rel'])
        #     print(df)
            start_time = time.time()
            while "next" in df['rel'].values:

                index = int(df[df['rel']=='next'].index.values)
    #             print(df['url'][index])
                url = df['url'][index]
                response = requests.get(url,auth=(client_id, client_secret))
                raw_list = response.json()

                updt_list = addMISCDetails(repo_id, org_id, issue_number, raw_list)

                repo_coll.insert_many(updt_list)

                print(response.headers.get('X-RateLimit-Remaining'))
                if int(response.headers.get('X-RateLimit-Remaining')) <= 2:
                    time.sleep(3600)

                link = response.headers.get('Link')
                clean_links = cleanLinks(link)
                df = pd.DataFrame(clean_links, columns =['url', 'rel'])

            print(time.time() - start_time)

        mongo_client.close()

In [61]:
mongo_client = pymongo.MongoClient("mongodb://127.0.0.1:27017/")
repodb = mongo_client["repo_collector"]
issues_coll = repodb["repo_issues"]

issue_result = list(issues_coll.find({},{"number": 1, "_id":0}))
# print(issue_result[0].get("number"))
mongo_client.close()

In [62]:
for issue in issue_result:
#     print(issue.get("number"))
    issue_number = str(issue.get("number"))
    comments_url = "https://api.github.com/repos/"+org_name+"/"+repo_name+"/issues/"+issue_number+"/comments?state=all&per_page=100"
    events_url = "https://api.github.com/repos/"+org_name+"/"+repo_name+"/issues/"+issue_number+"/events?state=all&per_page=100"
    
    populateToDB(comments_url, org_name, repo_name, issue_number, "repo_comments")
    populateToDB(events_url, org_name, repo_name, issue_number, "repo_events")

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
